In [ ]:
 pip install unidecode

In [832]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json
from unidecode import unidecode

# get 250 top imdb movies links

In [970]:
url = "https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=horror&sort=user_rating,desc&start=51"

In [971]:
page = requests.get(url)
page

<Response [200]>

In [972]:
soup = BeautifulSoup(page.content, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   Feature Film,
Rating Count at least 25,000,
Horror
(Sorted by IMDb Rating Descending) - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
  </script>
  <link href="https

### get div of movies

In [973]:
scraped_movies = soup.find_all('h3', class_='lister-item-header')
scraped_movies

[<h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">51.</span>
 <a href="/title/tt0109506/">The Crow</a>
 <span class="lister-item-year text-muted unbold">(1994)</span>
 </h3>,
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">52.</span>
 <a href="/title/tt0103873/">Braindead</a>
 <span class="lister-item-year text-muted unbold">(1992)</span>
 </h3>,
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">53.</span>
 <a href="/title/tt0082010/">An American Werewolf in London</a>
 <span class="lister-item-year text-muted unbold">(1981)</span>
 </h3>,
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">54.</span>
 <a href="/title/tt0075005/">The Omen</a>
 <span class="lister-item-year text-muted unbold">(1976)</span>
 </h3>,
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">55.</span>
 <a href="/title/tt0073582/">Profond

### get movies name

In [974]:
movies_name = []
for movie in scraped_movies:
    movie = movie.get_text().replace('\n', "")
    movie = movie.strip(" ")
    movies_name.append(movie)
movies_name

['51.The Crow(1994)',
 '52.Braindead(1992)',
 '53.An American Werewolf in London(1981)',
 '54.The Omen(1976)',
 '55.Profondo rosso(1975)',
 '56.The Wicker Man(1973)',
 '57.Gojira(1954)',
 '58.The Lighthouse(I) (2019)',
 '59.Gokseong(2016)',
 '60.[Rec](2007)',
 '61.Bhoot Police(2021)',
 '62.El orfanato(2007)',
 '63.El espinazo del diablo(2001)',
 '64.Tesis(1996)',
 '65.Army of Darkness(1992)',
 '66.Dracula(1992)',
 "67.Jacob's Ladder(I) (1990)",
 '68.A Nightmare on Elm Street(1984)',
 '69.The Evil Dead(1981)',
 '70.Nosferatu: Phantom der Nacht(1979)',
 '71.Invasion of the Body Snatchers(1978)',
 '72.Carrie(1976)',
 '73.The Rocky Horror Picture Show(1975)',
 '74.The Texas Chain Saw Massacre(1974)',
 '75.The Haunting(1963)',
 '76.Dracula(1931)',
 '77.Hereditary(2018)',
 '78.Doctor Sleep(2019)',
 '79.Split(IX) (2016)',
 '80.The Conjuring 2(2016)',
 '81.It(I) (2017)',
 '82.Sweeney Todd: The Demon Barber of Fleet Street(2007)',
 '83.Dawn of the Dead(2004)',
 '84.Sleepy Hollow(1999)',
 '85.Pi

### get movies links

In [975]:
links_regex = soup.find_all('a', {'href': re.compile(r'title')})
links_regex

[<a aria-disabled="false" class="ipc-list__item nav-link nav-link--hideXS nav-link--hideS nav-link--hideM NavLink-sc-19k0khm-0 LrpYY ipc-list__item--indent-one" href="https://www.imdb.com/search/title/?count=100&amp;groups=oscar_best_picture_winners&amp;sort=year%2Cdesc&amp;ref_=nv_ch_osc" role="menuitem" tabindex="-1"><span class="ipc-list-item__text" role="presentation">Best Picture Winners</span></a>,
 <a class="compact" href="/search/title/?title_type=feature&amp;num_votes=25000,&amp;genres=horror&amp;sort=user_rating,desc&amp;start=51&amp;view=simple"> Compact
 </a>,
 <a class="detailed" href="/search/title/?title_type=feature&amp;num_votes=25000,&amp;genres=horror&amp;sort=user_rating,desc&amp;start=51&amp;view=advanced"> <strong>Detailed</strong>
 </a>,
 <a class="lister-page-prev prev-page" href="/search/title/?title_type=feature&amp;num_votes=25000,&amp;genres=horror&amp;sort=user_rating,desc">« Previous</a>,
 <a class="lister-page-next next-page" href="/search/title/?title_ty

### clean the links and remove extra links

In [976]:
urls = []
pre_url = 'https://www.imdb.com/'
for movie in soup.find_all('a', {'href': re.compile(r'title')}):
    movie = pre_url + movie['href']
    urls.append(movie)

urls.remove(urls[0])

size = len(movies_name)
rgx = "^https:\/\/www\.imdb\.com\/\/title\/tt\d+\/$"
movies_link = []
i = len(urls)
for m in range(1, size*2, 2):
    if re.search(rgx, urls[m]):
        movies_link.append(urls[m])

movies_link

['https://www.imdb.com//title/tt0109506/',
 'https://www.imdb.com//title/tt0103873/',
 'https://www.imdb.com//title/tt0082010/',
 'https://www.imdb.com//title/tt0075005/',
 'https://www.imdb.com//title/tt0073582/',
 'https://www.imdb.com//title/tt0070917/',
 'https://www.imdb.com//title/tt0047034/',
 'https://www.imdb.com//title/tt7984734/',
 'https://www.imdb.com//title/tt5215952/',
 'https://www.imdb.com//title/tt1038988/',
 'https://www.imdb.com//title/tt10083640/',
 'https://www.imdb.com//title/tt0464141/',
 'https://www.imdb.com//title/tt0256009/',
 'https://www.imdb.com//title/tt0117883/',
 'https://www.imdb.com//title/tt0106308/',
 'https://www.imdb.com//title/tt0103874/',
 'https://www.imdb.com//title/tt0099871/',
 'https://www.imdb.com//title/tt0087800/',
 'https://www.imdb.com//title/tt0083907/',
 'https://www.imdb.com//title/tt0079641/',
 'https://www.imdb.com//title/tt0077745/',
 'https://www.imdb.com//title/tt0074285/',
 'https://www.imdb.com//title/tt0073629/',
 'https://

## crawl each link
### and store in the json file

In [977]:
f = open("horror2.json", 'x')
c = -1

for link in movies_link:
    c = c + 1
    print(c)
    url = link
    print(link)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    
    scraped_storyline = soup.find_all('div', class_='ipc-html-content ipc-html-content--base')
    storyline = scraped_storyline[0].get_text()
    storyline = unidecode(storyline)
    
    scraped_genre = soup.find_all('li', class_='ipc-inline-list__item')
    movies_genre = []
    for movie in soup.find_all('a', {'href': re.compile(r'genres')}):
        movie = movie.get_text().replace('\n', "")
        movie = movie.strip(" ")
        movies_genre.append(movie)

    x = len(movies_genre)
    genre = []
    
    for m in range(int(x/2)):
        genre.append(movies_genre[m])
    
    listToStr = '", "'.join([str(elem) for elem in genre])
    text = "{\n \"genre\":" + "[\"" + listToStr + "\" "]", \n \"storyline:\"" + storyline + "\" , \n}"
    f.write(text + '\n')
f.close()

0
https://www.imdb.com//title/tt0109506/
1
https://www.imdb.com//title/tt0103873/
2
https://www.imdb.com//title/tt0082010/
3
https://www.imdb.com//title/tt0075005/
4
https://www.imdb.com//title/tt0073582/
5
https://www.imdb.com//title/tt0070917/
6
https://www.imdb.com//title/tt0047034/
7
https://www.imdb.com//title/tt7984734/
8
https://www.imdb.com//title/tt5215952/
9
https://www.imdb.com//title/tt1038988/
10
https://www.imdb.com//title/tt10083640/
11
https://www.imdb.com//title/tt0464141/
12
https://www.imdb.com//title/tt0256009/
13
https://www.imdb.com//title/tt0117883/
14
https://www.imdb.com//title/tt0106308/
15
https://www.imdb.com//title/tt0103874/
16
https://www.imdb.com//title/tt0099871/
17
https://www.imdb.com//title/tt0087800/
18
https://www.imdb.com//title/tt0083907/
19
https://www.imdb.com//title/tt0079641/
20
https://www.imdb.com//title/tt0077745/
21
https://www.imdb.com//title/tt0074285/
22
https://www.imdb.com//title/tt0073629/
23
https://www.imdb.com//title/tt0072271/
2

## just for test
### store the storyline

In [707]:
url = 'https://www.imdb.com//title/tt11685912/'
page = requests.get(url)
print(page)
soup = BeautifulSoup(page.content, "html.parser")

scraped_storyline = soup.find_all('div', class_='ipc-html-content ipc-html-content--base')
storyline = scraped_storyline[0].get_text()
storyline = unidecode(storyline)

<Response [200]>


In [708]:
storyline = scraped_storyline[0].get_text()
unidecode(storyline)

"A rancher fighting for his land and family discovers an unfathomable mystery at the edge of Wyoming's wilderness."

### store the genre

In [709]:
scraped_genre = soup.find_all('li', class_='ipc-inline-list__item')
scraped_genre

[<li class="ipc-inline-list__item" role="presentation">TV Series</li>,
 <li class="ipc-inline-list__item" role="presentation"><a class="ipc-link ipc-link--baseAlt ipc-link--inherit-color sc-8c396aa2-1 WIUyh" href="/title/tt11685912/releaseinfo?ref_=tt_ov_rdat">2022– </a><span class="sc-8c396aa2-2 itZqyK">2022– </span></li>,
 <li class="ipc-inline-list__item" role="presentation"><a class="ipc-link ipc-link--baseAlt ipc-link--inherit-color sc-8c396aa2-1 WIUyh" href="/title/tt11685912/parentalguide/certificates?ref_=tt_ov_pg">TV-MA</a><span class="sc-8c396aa2-2 itZqyK">TV-MA</span></li>,
 <li class="ipc-inline-list__item" role="presentation"><a class="ipc-link ipc-link--baseAlt ipc-link--inherit-color" href="fullcredits/?ref_=tt_ql_cl">Cast &amp; crew</a></li>,
 <li class="ipc-inline-list__item" role="presentation"><a class="ipc-link ipc-link--baseAlt ipc-link--inherit-color" href="reviews/?ref_=tt_ql_urv">User reviews</a></li>,
 <li class="ipc-inline-list__item" role="presentation"><a cl

In [710]:
movies_genre = []
for movie in soup.find_all('a', {'href': re.compile(r'genres')}):
    movie = movie.get_text().replace('\n', "")
    movie = movie.strip(" ")
    movies_genre.append(movie)

x = len(movies_genre)
genre = []
for m in range(int(x/2)):
    genre.append(movies_genre[m])
    
genre

['Drama', 'Mystery', 'Thriller']

In [371]:
mk = open("test.json", "w")
storyline = replace_umlauts(storyline)
mk.write(storyline)
mk.close()

In [394]:
from unidecode import unidecode

city = "České Budějovice"
print(unidecode(city))

Ceske Budejovice
